In [21]:
import importlib
import utilities.functions as functions
import pandas as pd
importlib.reload(functions)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json



from utilities.functions import (
    retidos11,
    calcula_viabilidade_wide,
    


)

In [3]:
df = pd.read_parquet("../dados/gold/df_decil.parquet")
df.head()


,customer_id,is_target,decil_1,decil_12,num_pedidos_mes_1,num_pedidos_mes_12,total_amount_mes_1,total_amount_mes_12
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,decil 10,decil 7,8,2,273.00,100.00
1,000021924bf8192f64a7cc17ce7c0bc511cba907308f80...,target,decil 9,decil 8,3,2,173.90,119.90
2,000022ced2f3aa411b66ac41167b272ae528c4acdffeeb...,target,decil 6,decil 6,2,2,68.65,68.65
3,000032b594523c3f8868edee4f1577b157e115cd01ab31...,control,decil 7,decil 7,1,1,88.80,88.80
4,0000bb10fb47a1d6b2d73754ef383950ef536c77d07212...,control,decil 9,decil 9,2,2,188.19,188.19


In [4]:
df['decil_1_num'] = df['decil_1'].str.extract('(\d+)').astype(int)
df['decil_12_num'] = df['decil_12'].str.extract('(\d+)').astype(int)
df['delta_decil'] = df['decil_12_num'] - df['decil_1_num']

<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:1: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
/var/folders/s5/dkrzm08d6wl79n0by4mcn72m0000gn/T/ipykernel_7982/3781260405.py:1: SyntaxWarning: invalid escape sequence '\d'
  df['decil_1_num'] = df['decil_1'].str.extract('(\d+)').astype(int)
/var/folders/s5/dkrzm08d6wl79n0by4mcn72m0000gn/T/ipykernel_7982/3781260405.py:2: SyntaxWarning: invalid escape sequence '\d'
  df['decil_12_num'] = df['decil_12'].str.extract('(\d+)').astype(int)


In [ ]:
df[['segmento_mobilidade', 'direcao']] = (df['delta_decil'].apply(lambda x: pd.Series(functions.classifica(x))))

,customer_id,is_target,decil_1,decil_12,num_pedidos_mes_1,num_pedidos_mes_12,total_amount_mes_1,total_amount_mes_12,decil_1_num,decil_12_num,delta_decil,segmento_mobilidade,direcao
0,000009e29b12d8c5b9c9579e21384320c2df3bd935f19e...,control,decil 10,decil 7,8,2,273.00,100.00,10,7,-3,Alta Mobilidade,Upgrade
1,000021924bf8192f64a7cc17ce7c0bc511cba907308f80...,target,decil 9,decil 8,3,2,173.90,119.90,9,8,-1,Mobilidade Moderada,Upgrade
2,000022ced2f3aa411b66ac41167b272ae528c4acdffeeb...,target,decil 6,decil 6,2,2,68.65,68.65,6,6,0,Estável,Estável
3,000032b594523c3f8868edee4f1577b157e115cd01ab31...,control,decil 7,decil 7,1,1,88.80,88.80,7,7,0,Estável,Estável
4,0000bb10fb47a1d6b2d73754ef383950ef536c77d07212...,control,decil 9,decil 9,2,2,188.19,188.19,9,9,0,Estável,Estável
...,...,...,...,...,...,...,...,...,...,...,...,...,...
554371,ffffd512938c9c6a7fc101b6bc044c2bc542505ceff2a5...,control,decil 9,decil 5,6,2,187.40,53.30,9,5,-4,Alta Mobilidade,Upgrade
554372,ffffdfae8a70b042a8f86763f59024f61899d6f30dec60...,target,decil 1,decil 1,1,1,22.80,22.80,1,1,0,Estável,Estável
554373,ffffe50aaaed54af0eb463fcc6761b0e17a1d1cdd6b8e1...,control,decil 5,decil 5,1,1,55.60,55.60,5,5,0,Estável,Estável
554374,ffffe95dc11472a22c46b6d5cdc5c14cf40e0c88a601e0...,target,decil 9,decil 9,2,2,209.00,209.00,9,9,0,Estável,Estável


In [7]:
tabela_segmento = (
    df
    .groupby(['segmento_mobilidade', 'direcao', 'is_target'])
    .agg(
        clientes=('customer_id', 'nunique'),
        decil_inicial_medio=('decil_1_num', 'mean'),
        decil_final_medio=('decil_12_num', 'mean'),
        pedidos_medios_mes_1=('num_pedidos_mes_1', 'mean'),
        pedidos_medios_mes_12=('num_pedidos_mes_12', 'mean'),
        ticket_medio_mes_1=('total_amount_mes_1', 'mean'),
        ticket_medio_mes_12=('total_amount_mes_12', 'mean')
    )
    .reset_index()
)




In [8]:
tabela_segmento

,segmento_mobilidade,direcao,is_target,clientes,decil_inicial_medio,decil_final_medio,pedidos_medios_mes_1,pedidos_medios_mes_12,ticket_medio_mes_1,ticket_medio_mes_12
0,Alta Mobilidade,Upgrade,control,63005,8.041219,4.558289,4.109864,1.602587,168.126372,59.897590
1,Alta Mobilidade,Upgrade,target,94674,8.024495,4.534804,4.086412,1.596341,167.860583,59.595718
2,Estável,Estável,control,152564,5.224083,5.224083,2.859777,2.170997,152.255066,112.578641
3,Estável,Estável,target,179684,5.664121,5.664121,3.353203,2.478011,179.177332,128.823753
4,Mobilidade Moderada,Upgrade,control,25888,8.984433,7.984433,5.780632,3.185994,260.700879,142.219690
5,Mobilidade Moderada,Upgrade,target,38561,8.973393,7.973393,5.833874,3.200202,261.626572,141.997371


In [9]:
tabela_segmento.to_csv('../Resultados/tabela_segmento.csv', index=False)

Marcar segmentacao - baseada no amount

In [10]:

mapa_categorias = {
    ('Alta Mobilidade', 'Upgrade'): 1,
    ('Estável', 'Estável'): 2,
    ('Mobilidade Moderada', 'Upgrade'): 3
}

df['categoria_segmento'] = df.apply(
    lambda x: mapa_categorias.get((x['segmento_mobilidade'], x['direcao']), None),
    axis=1
)

resumo_categorias = (
    df
    .groupby(['categoria_segmento', 'is_target'])['customer_id']
    .nunique()
    .reset_index(name='num_clientes')
    .sort_values(['categoria_segmento', 'is_target'])
)




In [11]:
df11=df[df['categoria_segmento']==1]
df12=df[df['categoria_segmento']==2]
df13=df[df['categoria_segmento']==3]

In [19]:
df_1,clientes_retidos_1=retidos11(df11, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)
df_2,clientes_retidos_2=retidos11(df12, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_1)
df_3,clientes_retidos_3=retidos11(df13, mes0=12, mes1=1,pedidos=1)
print(clientes_retidos_3)

           retidos   base  taxa_retencao
is_target                               
control      63005  63005            1.0
target       94674  94674            1.0
           retidos   base  taxa_retencao
is_target                               
control      63005  63005            1.0
target       94674  94674            1.0
           retidos   base  taxa_retencao
is_target                               
control      25888  25888            1.0
target       38561  38561            1.0


In [22]:
resultados, agg = calcula_viabilidade_wide(df_1,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

{'pedidos_por_cliente_camp_control': 1.6025870962622013, 'pedidos_por_cliente_camp_target': 1.59634112850413, 'pedidos_por_cliente_seg_control': 4.109864296484406, 'pedidos_por_cliente_seg_target': 4.086412320172381, 'inc_pedidos_camp': -591.3307515276414, 'inc_pedidos_seg': -2220.2924053646234, 'inc_pedidos_total': -2811.623156892265, 'inc_gmv_camp': -28579.378687564447, 'inc_gmv_seg': -25163.240864374948, 'inc_gmv_total': -53742.619551939395, 'margem_incremental': -6449.114346232727, 'base_target_camp': 94674, 'custo_campanha': 946740.0, 'lucro_incremental': -953189.1143462327, 'roi': -1.006811917048221, 'coupon_value': 10.0, 'margin_rate': 0.12, 'mes_campanha': 12, 'mes_seguinte': 1}
           pedidos_campanha  gmv_campanha  pedidos_seguinte  gmv_seguinte  \
is_target                                                                   
control              100971    3773847.64            258942   10592802.04   
target               151132    5642165.03            386877   15892032.86

In [23]:
resultados, agg = calcula_viabilidade_wide(df_2,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

{'pedidos_por_cliente_camp_control': 2.1709970897459425, 'pedidos_por_cliente_camp_target': 2.4780113977872267, 'pedidos_por_cliente_seg_control': 2.8597768805222725, 'pedidos_por_cliente_seg_target': 3.3532034015271255, 'inc_pedidos_camp': 55165.5589260901, 'inc_pedidos_seg': 88660.851000236, 'inc_pedidos_total': 143826.4099263261, 'inc_gmv_camp': 2918986.7852843422, 'inc_gmv_seg': 4837500.541719672, 'inc_gmv_total': 7756487.3270040145, 'margem_incremental': 930778.4792404817, 'base_target_camp': 179684, 'custo_campanha': 1796840.0, 'lucro_incremental': -866061.5207595183, 'roi': -0.4819914520822768, 'coupon_value': 10.0, 'margin_rate': 0.12, 'mes_campanha': 12, 'mes_seguinte': 1}
           pedidos_campanha  gmv_campanha  pedidos_seguinte  gmv_seguinte  \
is_target                                                                   
control              331216   17175447.79            436299   23228641.88   
target               445259   23147567.32            602517   32195299.81   



In [24]:
resultados, agg = calcula_viabilidade_wide(df_3,
                                           mes_campanha=12,
                                           mes_seguinte=1,
                                           coupon_value=10.0,
                                           margin_rate=0.12)

print(resultados)
print(agg)

{'pedidos_por_cliente_camp_control': 3.1859935105067985, 'pedidos_por_cliente_camp_target': 3.200202276911906, 'pedidos_por_cliente_seg_control': 5.78063195302843, 'pedidos_por_cliente_seg_target': 5.8338736028629965, 'inc_pedidos_camp': 547.9042413473455, 'inc_pedidos_seg': 2053.0512592707178, 'inc_pedidos_total': 2600.9555006180635, 'inc_gmv_camp': -8572.81905941035, 'inc_gmv_seg': 35695.6431508831, 'inc_gmv_total': 27122.82409147275, 'margem_incremental': 3254.73889097673, 'base_target_camp': 38561, 'custo_campanha': 385610.0, 'lucro_incremental': -382355.2611090233, 'roi': -0.9915595060009421, 'coupon_value': 10.0, 'margin_rate': 0.12, 'mes_campanha': 12, 'mes_seguinte': 1}
           pedidos_campanha  gmv_campanha  pedidos_seguinte  gmv_seguinte  \
is_target                                                                   
control               82479    3681783.33            149649    6749024.35   
target               123403    5475560.64            224960   10088582.23   

    